In [1]:
#Importações
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
import plotly.express as px

In [4]:
import plotly.graph_objs as go

In [5]:
! pip install unidecode
from unidecode import unidecode


In [6]:
! pip install jupyter-dash -q

In [7]:
! pip install dash-cytoscape -q

In [8]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input

In [9]:
from jupyter_dash import JupyterDash

In [10]:
import urllib.request, urllib.parse, urllib.error 

In [11]:
from urllib.error import HTTPError

In [12]:
#Criação do DataFrame, verificando a disponibildade dos dados mais atualizados da própria ANP. Em caso de erro, utiliza-se uma base antiga.
sucesso = 'Falso' 
try:
   df = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/qus/ultimas-4-semanas-gasolina-etanol-2.csv',sep=';',parse_dates=['Data da Coleta'],dayfirst=True)
   sucesso = 'Verdadeiro'
except UnicodeDecodeError:   
  try:
    df = pd.read_csv('https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/arquivos/shpc/qus/ultimas-4-semanas-gasolina-etanol-2.csv',sep=';',parse_dates=['Data da Coleta'],dayfirst=True,encoding='latin-1')
    sucesso = 'Verdadeiro' 
  except Exception:
    pass
except HTTPError:
  pass
except Exception:
  pass      
if sucesso == 'Falso':
  df = pd.read_csv('https://raw.githubusercontent.com/lianasilveira/dashboard-precos-combustiveis-brasil/main/ultimas-4-semanas-gasolina-etanol-2.csv',sep=';',parse_dates=['Data da Coleta'],dayfirst=True)
  print('Não foi possível realizar a leitura dos dados atualizados da ANP. A base utilizada representa o período de 27/09/2021 a 22/10/2021')
else:
  print('Dados atualizados da ANP referentes as 4 últimas semanas')

Dados atualizados da ANP referentes as 4 últimas semanas


In [13]:
#Tratamento dos dados
df.dtypes

Região - Sigla               object
Estado - Sigla               object
Município                    object
Revenda                      object
CNPJ da Revenda              object
Nome da Rua                  object
Número Rua                   object
Complemento                  object
Bairro                       object
Cep                          object
Produto                      object
Data da Coleta       datetime64[ns]
Valor de Venda               object
Valor de Compra             float64
Unidade de Medida            object
Bandeira                     object
dtype: object

In [14]:
df.drop(['Valor de Compra'], axis=1, inplace=True)

In [15]:
df['Valor de Venda'] = df['Valor de Venda'].str.replace(',', '.').astype(float)

In [16]:
df['Bairro'] = df['Bairro'].astype(str).map(unidecode)

In [17]:
df['Bairro'].replace(['nan'], 'Nao Informado', inplace=True)

In [96]:
#Dashboard
app = JupyterDash(__name__)

app.layout=html.Div([
    html.Div([                 
                     html.H1('"Abastece Aqui!" Informações sobre os preços dos combustíveis na sua Região e Município - em R$/litro',
                             style={'font-size':'20px','width': '100%'}),
       
            html.Div([html.Label('Escolha um período:',style={'float':'left','font-size':'14px'}),
                     
                     dcc.DatePickerSingle(id='dat1', 
                                  min_date_allowed=df['Data da Coleta'].min(),
                                  max_date_allowed=df['Data da Coleta'].max(),
                                  display_format='YYYY-MM-DD',
                                  date=df['Data da Coleta'].min()),
                                   

                     dcc.DatePickerSingle(id='dat2', 
                                  min_date_allowed=df['Data da Coleta'].min(),
                                  max_date_allowed=df['Data da Coleta'].max(),
                                  display_format='YYYY-MM-DD',
                                  date=df['Data da Coleta'].max())
                     ],
                     className='row')

     ]),


    html.Div([                 
                     
                     dcc.Dropdown(id='prods', 
                                  options=[{'label':x, 'value':x} 
                                            for x in sorted(df['Produto'].unique())],
                                  value='GASOLINA'
                                  ),
                     
                     dcc.Dropdown(id='regs',
                                  options=[{'label':x, 'value':x}
                                           for x in sorted(df['Região - Sigla'].unique())],
                                  value='SE'
                                  ),
                     
                     dcc.Graph(id='my-graph', 
                               figure={},
                               style={'width': '70vh', 'height': '70vh'}
                              ),
     ],

style={'width': '30%', 'display': 'inline-block', 'float':'left'}),              
     
     html.Div([
        
                     dcc.Dropdown(id='ests',
                                  value='SP'),

                     dcc.Dropdown(id='cids',
                                  value='SAO PAULO'),
               
                     dcc.Graph(id='my-graph2', 
                               figure={},
                               style={'height': 400}
                              ),
               
                     dcc.Graph(id='my-graph3', 
                              figure={},
                              style={'height': 500}
                              ),
],

style={'width': '66%', 'float': 'right', 'display': 'inline-block'})
     
])



@app.callback(Output(component_id='my-graph', component_property='figure'),
              [Input('dat1', 'date'),
               Input('dat2', 'date'),               
               Input(component_id='prods', component_property='value'),
               Input(component_id='regs', component_property='value')]
)

def grafico_regiao(data_inicial, data_final, valor_produto, valor_regiao):
    mediaNacional = df[(df['Produto'] == valor_produto)]
    mediaNacional = mediaNacional['Valor de Venda'].mean()
    dff = df.loc[df['Data da Coleta'] >= data_inicial]
    dff = dff.loc[df['Data da Coleta'] <= data_final] 
    dff = dff[(dff['Produto'] == valor_produto) & (dff['Região - Sigla'] == valor_regiao)]
    dff = dff.groupby('Estado - Sigla')['Valor de Venda'].mean()
    fig = px.bar(data_frame=dff.reset_index(),x='Valor de Venda',y='Estado - Sigla')
    fig.add_vline(x=mediaNacional, line_width=2, line_color='orange')
    fig.update_layout(title_text='Média dos Preços de Combustíveis, e comparativo com a Média Nacional: R$ {:.3f}'.format(mediaNacional), font=dict(size=7))
    return fig

@app.callback(Output(component_id='ests', component_property='options'),
              [Input('dat1', 'date'),
               Input('dat2', 'date'),
               Input(component_id='prods', component_property='value'),
               Input(component_id='regs', component_property='value')]
)

def iterativo_estado(data_inicial, data_final, valor_produto, valor_regiao):
    dff1 = df.loc[df['Data da Coleta'] >= data_inicial]
    dff1 = dff1.loc[df['Data da Coleta'] <= data_final] 
    dff1 = dff1[(dff1['Produto'] == valor_produto) & (dff1['Região - Sigla'] == valor_regiao)] 
    return [{'label': x, 'value': x} for x in sorted(dff1['Estado - Sigla'].unique())]

@app.callback(Output(component_id='cids', component_property='options'),
              [Input('dat1', 'date'),
               Input('dat2', 'date'),               
               Input(component_id='prods', component_property='value'),
              Input(component_id='ests', component_property='value')]
)

def iterativo_municipio(data_inicial, data_final, valor_produto, valor_estado):
    dff2 = df.loc[df['Data da Coleta'] >= data_inicial]
    dff2 = dff2.loc[df['Data da Coleta'] <= data_final] 
    dff2 = dff2[(dff2['Produto'] == valor_produto) & (dff2['Estado - Sigla'] == valor_estado)] 
    return [{'label': x, 'value': x} for x in sorted(dff2['Município'].unique())]


@app.callback(Output(component_id='my-graph2', component_property='figure'),
              [Input('dat1', 'date'),
               Input('dat2', 'date'),               
               Input(component_id='prods', component_property='value'),
               Input(component_id='ests', component_property='value'),
               Input(component_id='cids', component_property='value')]
)

def grafico_bairro(data_inicial, data_final, valor_produto, valor_estado, valor_municipio):
    dff3 = df.loc[df['Data da Coleta'] >= data_inicial]
    dff3 = dff3.loc[df['Data da Coleta'] <= data_final] 
    dff3 = dff3[(dff3['Produto'] == valor_produto) & (dff3['Estado - Sigla'] == valor_estado) & (dff3['Município'] == valor_municipio)]
    mediaCidade = dff3['Valor de Venda'].mean()
    fig1 = px.bar(data_frame=dff3.groupby('Bairro')['Valor de Venda'].mean().reset_index(),x='Bairro',y='Valor de Venda',range_y=[(dff3['Valor de Venda'].min()), dff3['Valor de Venda'].max()])
    return fig1  

@app.callback(Output(component_id='my-graph3', component_property='figure'),
              [Input('dat1', 'date'),
               Input('dat2', 'date'),               
               Input(component_id='prods', component_property='value'),
               Input(component_id='ests', component_property='value'),
               Input(component_id='cids', component_property='value')]
)

def grafico_revenda(data_inicial, data_final, valor_produto, valor_estado, valor_municipio):
    dff4 = df.loc[df['Data da Coleta'] >= data_inicial]
    dff4 = dff4.loc[df['Data da Coleta'] <= data_final]     
    dff4 = dff4[(dff4['Produto'] == valor_produto) & (dff4['Estado - Sigla'] == valor_estado) & (dff4['Município'] == valor_municipio)]
    dff4 = dff4.loc[dff4.groupby('Revenda')['Data da Coleta'].idxmax(),:]
    fig2 = px.scatter(data_frame=dff4,x=sorted(dff4['Bairro']),y='Valor de Venda',hover_name='Revenda',hover_data=['Nome da Rua', 'Número Rua', 'Bandeira','Data da Coleta'])
    return fig2  


app.run_server(port='8050')

Dash app running on:


<IPython.core.display.Javascript object>